In [21]:
import os
import json

import numpy as np
import pandas as pd
import py7zr
import scipy.stats
from scipy.stats import expon
from urllib.request import urlopen

In [5]:
def grouped_weighted_avg(values, weights, by):
    return (values * weights).groupby(by).sum()  / weights.groupby(by).sum()

In [4]:
curr_dir = os.getcwd()
parent_dir = os.path.dirname(curr_dir)

# Data reading
with py7zr.SevenZipFile(parent_dir + '/data/globalterrorismdb_0221dist.7z') as z:
    # open the csv file in the dataset
    targetPath = os.path.expanduser("~/Desktop/GT Dataset")
    z.extract(path = targetPath)
    df = pd.read_excel(targetPath + '/globalterrorismdb_0221dist.xlsx')
    
# We filtered out doubted attacks to be able to have exact attacks
df = df[df['doubtterr'] != 1]

_df = df[['eventid','iyear','imonth','iday','country','country_txt',
         'region','region_txt', 'provstate', 'city','latitude','longitude',
         'gname','nkill', 'nwound']]

In [10]:
# Geojson for polygons of map
with urlopen('https://raw.githubusercontent.com/johan/world.geo.json/master/countries.geo.json') as response:
    countries = json.load(response)
                          
countries_df = pd.json_normalize(countries,  record_path =['features'])

In [11]:
# Updating the countries to be able to merge -- TODO: !!!!
_df.loc[_df['country_txt'] == 'Bahamas','country_txt']  = 'The Bahamas'
_df.loc[_df['country_txt'] == 'United States','country_txt'] = 'United States of America'
_df.loc[_df['country_txt'] == 'Czechoslovakia','country_txt'] = 'Czech Republic'
_df.loc[_df['country_txt'] == 'East Germany (GDR)','country_txt']  = 'Germany'
_df.loc[_df['country_txt'] == 'West Germany (FRG)','country_txt']  = 'Germany'

_df.loc[_df['country_txt'] == 'West Bank and Gaza Strip','country_txt'] = 'West Bank'
_df.loc[_df['country_txt'] == 'South Vietnam','country_txt'] = 'Vietnam'
_df.loc[_df['country_txt'] == 'North Yemen','country_txt'] = 'Yemen'
_df.loc[_df['country_txt'] == 'South Yemen','country_txt'] = 'Yemen'
#_df.loc[_df['country_txt'] == 'Andorra','country_txt'] = 'Spain'
_df.loc[_df['country_txt'] == 'Bahrain','country_txt'] = 'Iran'
_df.loc[_df['country_txt'] == 'Tanzania','country_txt'] = 'United Republic of Tanzania'

for ind in _df.index:
    if _df.loc[ind,'country_txt'] in ['Soviet Union', 'Yugoslavia']:
        _df.loc[ind,'country_txt'] = _df.loc[ind,'provstate']


_df.loc[_df['country_txt'] == 'Bosnia-Herzegovina','country_txt'] = 'Bosnia and Herzegovina'
_df.loc[_df['country_txt'].isin(['Central Serbia','Serbia','Serbia-Montenegro',
                                 'Republika Srpska','Belgrade','Kosovo and Metohija']),'country_txt'] = 'Republic of Serbia'  
_df.loc[_df['country_txt'] == 'Yugoslavia','country_txt'] = 'Republic of Serbia'
_df.loc[_df['country_txt'] == 'International','country_txt'] = 'United Arab Emirates'
_df.loc[_df['country_txt'] == 'Hong Kong','country_txt'] = 'China' 
_df.loc[_df['country_txt'] == 'Kygyzstan','country_txt'] = 'Kyrgyzstan'
_df.loc[_df['country_txt'] == 'Maldives','country_txt'] = 'India'

C:\Users\Esra\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [12]:
# Merging locations and main df
df_merged = pd.merge(_df, countries_df , left_on = 'country_txt', right_on = 'properties.name', how = 'left' )

# Determine ig organized or unorganized
df_merged['isOrganized'] = ['unorganized' if i == 'Unknown' else 'organized' for i in df_merged['gname']]


In [50]:
def create_yearly_file(df):
    weights =  np.linspace(expon.ppf(0.01), expon.ppf(0.99), len(df.iyear.unique()))
    weights = weights/ np.sum(weights)

    years = df_merged.iyear.unique()
    weighted_years = pd.DataFrame({'years': years, 'weights': weights})

    df = pd.merge(df, weighted_years, left_on = 'iyear', right_on = 'years', how = 'inner')
    df['total_kills_injured'] = _df['nkill'] + _df['nwound']

    df_map = df.groupby(by=['country','country_txt', 'iyear','id']).agg({
                                                                'nkill': np.sum,
                                                                'nwound': np.sum,
                                                                'total_kills_injured':np.sum,
                                                                'eventid': 'count', 
                                                                'weights': np.mean}).reset_index()
    
    calculated_indexes =  pd.DataFrame(grouped_weighted_avg(values=df_map.eventid, 
                                                            weights=df_map.weights, 
                                                            by=df_map.country)).reset_index().rename(columns={0: 'calculated_index'})
    
    df_map = df_map.groupby(by=['country','country_txt','id']).agg({
                                                                'nkill': np.sum,
                                                                'nwound': np.sum,
                                                                'total_kills_injured':np.sum}).reset_index()
    
    df_map = pd.merge(df_map, calculated_indexes, on = 'country', how = 'left')
    
    df_map.to_excel(parent_dir+'/data/'+'aggregated_map_data.xlsx', engine='openpyxl', encoding='utf-8')
    
    return df_map

In [51]:
def create_organized_nonorganized_data(df):
    df_h = df.groupby(by=['country','country_txt',
                          'iyear', 'isOrganized','id']).agg({'eventid':'count'}).unstack(fill_value=0).stack().sort_values(by='country').reset_index()
    
    df_x = df_h.pivot_table(index=['country','country_txt','iyear','id'], columns='isOrganized')
    countries = df_h.drop_duplicates(subset=['country','country_txt','iyear','id']).reset_index()[['country','country_txt','iyear','id']]
    countries['organized'] = df_x['eventid']['organized'].reset_index()['organized']
    countries['unorganized'] = df_x['eventid']['unorganized'].reset_index()['unorganized']
    countries['Total'] = countries['organized'] +countries['unorganized']
    countries = countries.sort_values(by='Total', ascending= False)
    
    countries.to_excel(parent_dir+'/data/'+'aggregated_attacktype_data.xlsx', engine='openpyxl', encoding='utf-8')
    
    return countries

In [70]:
def create_yearly_aggregated_data(df):
    _df = df.groupby(by=[ 'iyear', 'country', 'country_txt', 'id','isOrganized']) \
    .agg({'nkill':'sum',
          'nwound':'sum',
          'eventid':'nunique'}) \
    .unstack(fill_value=0) \
    .stack() \
    .sort_values(by='country') \
    .rename(columns={'eventid':'total_attacks'}) \
    .reset_index()
    
    _df['total_kill_injured'] = _df['nkill'] +_df['nwound']
    
    _df.to_excel(parent_dir+'/data/'+'aggregated_yearly_data.xlsx', engine='openpyxl', encoding='utf-8')
    
    return _df

In [52]:
create_yearly_file(df_merged)

,country,country_txt,id,nkill,nwound,total_kills_injured,calculated_index
0,4,Afghanistan,AFG,45461.0,51312.0,65114.0,571.889395
1,5,Albania,ALB,40.0,121.0,316.0,3.474904
2,6,Algeria,DZA,9742.0,8059.0,9086.0,65.668455
3,8,Angola,AGO,2304.0,2101.0,1045.0,14.131636
4,11,Argentina,ARG,299.0,674.0,1157.0,11.708865
...,...,...,...,...,...,...,...
191,603,United Kingdom,GBR,2567.0,5579.0,10418.0,79.024647
192,1001,Republic of Serbia,SRB,2.0,8.0,19.0,2.134499
193,1002,Montenegro,MNE,2.0,1.0,1.0,1.573993
194,1003,Kosovo,CS-KM,66.0,297.0,356.0,7.394328


In [53]:
create_organized_nonorganized_data(df_merged)

,country,country_txt,iyear,id,organized,unorganized,Total
256616,95,Iraq,2001,MOZ,999.0,2372.0,3371.0
256954,95,Iraq,1994,BOL,1025.0,1977.0,3002.0
256447,95,Iraq,1997,DZA,409.0,2096.0,2505.0
256785,95,Iraq,2000,GAB,788.0,1632.0,2420.0
257123,95,Iraq,1995,ETH,837.0,1182.0,2019.0
...,...,...,...,...,...,...,...
638646,1004,South Sudan,2018,CHL,NaN,0.0,NaN
638647,1004,South Sudan,2018,CHN,NaN,0.0,NaN
638648,1004,South Sudan,2018,CIV,NaN,0.0,NaN
638649,1004,South Sudan,2018,CMR,NaN,0.0,NaN


In [71]:
create_yearly_aggregated_data(df_merged)

,iyear,country,country_txt,id,isOrganized,nkill,nwound,total_attacks,total_kill_injured
0,2017,4,Afghanistan,AFG,organized,4235.0,4292.0,815,8527.0
1,2016,4,Afghanistan,AFG,organized,4112.0,4148.0,905,8260.0
2,2013,4,Afghanistan,AFG,organized,2411.0,2325.0,655,4736.0
3,2013,4,Afghanistan,AFG,unorganized,711.0,1391.0,496,2102.0
4,1997,4,Afghanistan,AFG,organized,0.0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...
7567,2016,1004,South Sudan,SSD,organized,431.0,119.0,24,550.0
7568,2016,1004,South Sudan,SSD,unorganized,41.0,43.0,23,84.0
7569,2017,1004,South Sudan,SSD,unorganized,71.0,99.0,16,170.0
7570,2013,1004,South Sudan,SSD,unorganized,32.0,4.0,6,36.0
